In [75]:
import os
import warnings
import pandas as pd
import csv
from census import Census
from us import states
from apikey import apikey
import requests

#ignore future warnings error
warnings.simplefilter(action='ignore', category=FutureWarning)

c = Census(apikey, year=2017)

In [76]:
# Open and read Texas Graduation Rate data
file = "C:/Users/May 2018/Desktop/SMU Bootcamp Doc/GroupProject/Team6/County_Grad_Data_2017.csv"
HighSchool2017_data = pd.read_csv(file)
HighSchool2017_data.head()

,COUNTY,CNTYNAME,CNTY_ALLD,CNTY_ALLN_CMP2,CNTY_ALLN_GRAD,CNTY_ALLN_CONT,CNTY_ALLN_TXCHSE,CNTY_ALLN_DROP,CNTY_ALLR_CMP2,CNTY_ALLR_GRAD,...,CNTY_TTLN_CONT,CNTY_TTLN_TXCHSE,CNTY_TTLN_DROP,CNTY_TTLR_CMP2,CNTY_TTLR_GRAD,CNTY_TTLR_CONT,CNTY_TTLR_TXCHSE,CNTY_TTLR_DROP,CNTY_TTLR_RAED,CNTY_TTLR_RECADV
0,1,ANDERSON COUNTY,<600,-,-,-,-,-,99.3,96.8,...,-,-,-,99.3,97.6,1.5,0.2,0.7,86.9,88.3
1,2,ANDREWS COUNTY,<250,-,-,-,0,-,97.1,95.8,...,0,0,5,97.8,97.8,0.0,0.0,2.2,-,-
2,3,ANGELINA COUNTY,"1,260","1,203","1,164",32,7,57,95.5,92.4,...,-,-,-,97.4,94.7,2.3,0.4,2.6,79.2,70.5
3,4,ARANSAS COUNTY,<250,-,-,-,-,-,91.0,88.7,...,-,-,-,91.0,88.7,1.4,0.9,9.0,82.2,82.7
4,5,ARCHER COUNTY,<200,-,-,-,0,0,100.0,99.4,...,0,0,0,.,.,.,.,.,.,.


In [77]:
# Create TX graduation data frame
HighSchool2017_dfw = HighSchool2017_data.loc[HighSchool2017_data['CNTYNAME'].isin(['DENTON COUNTY','COLLIN COUNTY','HUNT COUNTY','ROCKWALL COUNTY','KAUFMAN COUNTY','ELLIS COUNTY','JOHNSON COUNTY','DALLAS COUNTY','SOMMERVILLE COUNTY','HOOD','PARKER COUNTY','WISE COUNTY','TARRANT COUNTY'])]
HighSchool2017_dfw = pd.DataFrame(HighSchool2017_dfw)
HighSchool2017_dfw_df = HighSchool2017_dfw.filter(items=['CNTYNAME','CNTY_ALLR_GRAD','CNTY_AAR_GRAD','CNTY_ASR_GRAD',
                                                         'CNTY_HSR_GRAD','CNTY_MUR_GRAD','CNTY_NAR_GRAD','CNTY_PIR_GRAD',
                                                         'CNTY_WHR_GRAD'])
HighSchool2017_dfw_df.rename(columns={'CNTYNAME':'County Name','CNTY_ALLR_GRAD':'All Students','CNTY_AAR_GRAD':'African American',
                             'CNTY_ASR_GRAD':'Asian','CNTY_HSR_GRAD':'Hispanic','CNTY_MUR_GRAD':'Multiracial','CNTY_NAR_GRAD':'American Indian',
                                      'CNTY_PIR_GRAD':'Pacific Islander','CNTY_WHR_GRAD':'White'},inplace=True)
HighSchool2017_dfw_df

,County Name,All Students,African American,Asian,Hispanic,Multiracial,American Indian,Pacific Islander,White
42,COLLIN COUNTY,96.8,95.8,98.6,94.4,98.6,96.8,96.4,97.4
56,DALLAS COUNTY,84.0,80.8,93.5,84.0,81.3,74.4,92.5,87.1
60,DENTON COUNTY,94.5,92.1,96.7,91.4,96.7,95.7,100.0,95.9
69,ELLIS COUNTY,93.1,92.5,100.0,92.0,90.2,87.5,100.0,94.0
115,HUNT COUNTY,93.7,91.7,100.0,95.2,100.0,92.3,100.0,93.0
125,JOHNSON COUNTY,94.7,95.7,91.7,95.2,92.2,100.0,100.0,94.4
128,KAUFMAN COUNTY,95.5,97.0,93.8,94.4,95.1,80.0,100.0,95.8
181,PARKER COUNTY,96.3,97.1,100.0,97.0,97.1,100.0,100.0,96.1
196,ROCKWALL COUNTY,97.0,96.9,97.6,95.2,100.0,100.0,100.0,97.4
217,TARRANT COUNTY,90.5,87.7,94.1,88.2,92.1,90.0,95.6,93.9


In [78]:
# Open and read TX crime data 
TexasCrime2017_data = pd.read_csv("C:/Users/May 2018/Desktop/SMU Bootcamp Doc/GroupProject/Team6/County_crime_data_2017.csv")
TexasCrime2017_data.head()

,Agency Name,Number of Incidents
0,ANDERSON COUNTY,268
1,ANDERSON CO SO,105
2,PALESTINE PD,151
3,FRANKSTON PD,12
4,ANDREWS COUNTY,42


In [79]:
# Create TX crime data frame
DFWCrime2017 = TexasCrime2017_data.loc[TexasCrime2017_data['Agency Name'].isin(['DENTON COUNTY','COLLIN COUNTY','HUNT COUNTY','ROCKWALL COUNTY','KAUFMAN COUNTY','ELLIS COUNTY','JOHNSON COUNTY','DALLAS COUNTY','SOMMERVILLE COUNTY','HOOD','PARKER COUNTY','WISE COUNTY','TARRANT COUNTY'])]
DFWCrime2017_df = pd.DataFrame(DFWCrime2017)
DFWCrime2017_df.columns = ["County Name","Family Violence Incidents"]
DFWCrime2017_df


,County Name,Family Violence Incidents
210,COLLIN COUNTY,"3,227"
264,DALLAS COUNTY,"21,928"
304,DENTON COUNTY,"2,249"
356,ELLIS COUNTY,623
603,HUNT COUNTY,634
639,JOHNSON COUNTY,925
657,KAUFMAN COUNTY,682
855,PARKER COUNTY,520
906,ROCKWALL COUNTY,382
978,TARRANT COUNTY,"13,827"


In [80]:
# Merge TX graduation and crime data
HSGradCrime_2017_df = pd.merge(HighSchool2017_dfw_df,DFWCrime2017_df,on='County Name')
HSGradCrime_2017_df

,County Name,All Students,African American,Asian,Hispanic,Multiracial,American Indian,Pacific Islander,White,Family Violence Incidents
0,COLLIN COUNTY,96.8,95.8,98.6,94.4,98.6,96.8,96.4,97.4,"3,227"
1,DALLAS COUNTY,84.0,80.8,93.5,84.0,81.3,74.4,92.5,87.1,"21,928"
2,DENTON COUNTY,94.5,92.1,96.7,91.4,96.7,95.7,100.0,95.9,"2,249"
3,ELLIS COUNTY,93.1,92.5,100.0,92.0,90.2,87.5,100.0,94.0,623
4,HUNT COUNTY,93.7,91.7,100.0,95.2,100.0,92.3,100.0,93.0,634
5,JOHNSON COUNTY,94.7,95.7,91.7,95.2,92.2,100.0,100.0,94.4,925
6,KAUFMAN COUNTY,95.5,97.0,93.8,94.4,95.1,80.0,100.0,95.8,682
7,PARKER COUNTY,96.3,97.1,100.0,97.0,97.1,100.0,100.0,96.1,520
8,ROCKWALL COUNTY,97.0,96.9,97.6,95.2,100.0,100.0,100.0,97.4,382
9,TARRANT COUNTY,90.5,87.7,94.1,88.2,92.1,90.0,95.6,93.9,"13,827"


In [81]:
# Clean county names to match name style of census
County_Names = ['Collin County, Texas','Dallas County, Texas','Denton County, Texas','Ellis County, Texas','Hunt County, Texas','Johnson County, Texas','Kaufman County, Texas','Parker County, Texas','Rockwall County, Texas','Tarrant County, Texas','Wise County, Texas']
HSGradCrime_2017_df['County'] = County_Names
New_HSGradCrime_2017_df = HSGradCrime_2017_df[['County','All Students','Family Violence Incidents']] 
New_HSGradCrime_2017_df

,County,All Students,Family Violence Incidents
0,"Collin County, Texas",96.8,"3,227"
1,"Dallas County, Texas",84.0,"21,928"
2,"Denton County, Texas",94.5,"2,249"
3,"Ellis County, Texas",93.1,623
4,"Hunt County, Texas",93.7,634
5,"Johnson County, Texas",94.7,925
6,"Kaufman County, Texas",95.5,682
7,"Parker County, Texas",96.3,520
8,"Rockwall County, Texas",97.0,382
9,"Tarrant County, Texas",90.5,"13,827"


In [82]:
# Get DFW counties from API
census_data = c.acs5.get('NAME', geo={'for': 'county:121,085,231,397,257,139,251,113,221,367,497,439', 
                                      'in': 'state:%s' % states.TX.fips})

county_pd = pd.DataFrame(census_data)
county_pd

,NAME,county,state
0,"Hunt County, Texas",231,48
1,"Parker County, Texas",367,48
2,"Collin County, Texas",085,48
3,"Rockwall County, Texas",397,48
4,"Tarrant County, Texas",439,48
5,"Ellis County, Texas",139,48
6,"Wise County, Texas",497,48
7,"Kaufman County, Texas",257,48
8,"Denton County, Texas",121,48
9,"Dallas County, Texas",113,48


In [83]:
# Build census data frame
baseUrl = "https://api.census.gov/data/2017/acs/acs5/subject?get="
baseUrl2 = "https://api.census.gov/data/2017/acs/acsse?get="

#census ids
ids = ["K202002_001E", "K202503_002E", 
       "K202503_003E", "K201001_001E",
       "K201501_001E",  "K200101_003E",
       "K200101_002E"]

columns = ['County', 'Median Earnings', '# of Home Owners',
          '# of Renters', '# Married', "Education", 
           "Female Population", "Male Population" ]

censusDf = pd.DataFrame(columns=columns)


In [84]:
# get data for each id by county and populate dataframe
count = 0

for countyid in county_pd['county']: 
    
    for item in ids:
        targetUrl = (baseUrl2 + item + ",NAME" + "&for=county:" +
                           countyid + "&in=state:48" + "&key=" + apikey)
        resp = requests.get(targetUrl).json()
        
        if resp[0][0] == 'K202002_001E':
            censusDf.set_value(count, "Median Earnings", resp[1][0])
        if  resp[0][0] == 'K202503_002E':   
            censusDf.set_value(count, "# of Home Owners", resp[1][0])
        if  resp[0][0] == 'K202503_003E':     
            censusDf.set_value(count, "# of Renters", resp[1][0])
        if  resp[0][0] == 'K201001_001E':      
            censusDf.set_value(count, "# Married", resp[1][0])
        if  resp[0][0] == 'K201501_001E':       
            censusDf.set_value(count, "Education", resp[1][0])
        if resp[0][0] == 'K200101_003E':
            censusDf.set_value(count, "Female Population", resp[1][0])
        if resp[0][0] == 'K200101_002E':
            censusDf.set_value(count, "Male Population", resp[1][0])    
    censusDf.set_value(count, "County", resp[1][1])   
    count += 1

In [85]:
censusDf.head(12)

,County,Median Earnings,# of Home Owners,# of Renters,# Married,Education,Female Population,Male Population
0,"Hunt County, Texas",33664,64565,26438,75087,61557,48201,45671
1,"Parker County, Texas",35129,106949,24700,106767,90367,66140,67323
2,"Collin County, Texas",50212,665040,301277,760920,635534,492767,476836
3,"Rockwall County, Texas",50215,81035,15124,75858,63099,49144,47644
4,"Tarrant County, Texas",36216,1300652,728517,1600866,1317559,1047730,1006745
5,"Ellis County, Texas",37201,130247,41512,135260,111660,87970,85650
6,"Wise County, Texas",30706,50042,15105,52852,44568,33054,33127
7,"Kaufman County, Texas",36480,97537,23768,94327,78533,63092,59791
8,"Denton County, Texas",43250,579415,244764,662965,547898,424882,411328
9,"Dallas County, Texas",32275,1392954,1194085,2037627,1676011,1326753,1291395


In [86]:
# Merge all three data sets by county name
Merged_3_df = pd.merge(New_HSGradCrime_2017_df,censusDf,on='County')
Merged_3_df

,County,All Students,Family Violence Incidents,Median Earnings,# of Home Owners,# of Renters,# Married,Education,Female Population,Male Population
0,"Collin County, Texas",96.8,"3,227",50212,665040,301277,760920,635534,492767,476836
1,"Dallas County, Texas",84.0,"21,928",32275,1392954,1194085,2037627,1676011,1326753,1291395
2,"Denton County, Texas",94.5,"2,249",43250,579415,244764,662965,547898,424882,411328
3,"Ellis County, Texas",93.1,623,37201,130247,41512,135260,111660,87970,85650
4,"Hunt County, Texas",93.7,634,33664,64565,26438,75087,61557,48201,45671
5,"Johnson County, Texas",94.7,925,37424,124558,39913,131321,109599,83170,84131
6,"Kaufman County, Texas",95.5,682,36480,97537,23768,94327,78533,63092,59791
7,"Parker County, Texas",96.3,520,35129,106949,24700,106767,90367,66140,67323
8,"Rockwall County, Texas",97.0,382,50215,81035,15124,75858,63099,49144,47644
9,"Tarrant County, Texas",90.5,"13,827",36216,1300652,728517,1600866,1317559,1047730,1006745


In [87]:
# Caluate new metrics ... just an example - need more
# Home Ownership Ratio (# of Home Owners / # of Renters)
Merged_3_df["Home Ownership Ratio"] = Merged_3_df["# of Home Owners"].astype(int) / Merged_3_df["# of Renters"].astype(int)
Merged_3_df


,County,All Students,Family Violence Incidents,Median Earnings,# of Home Owners,# of Renters,# Married,Education,Female Population,Male Population,Home Ownership Ratio
0,"Collin County, Texas",96.8,"3,227",50212,665040,301277,760920,635534,492767,476836,2.207404
1,"Dallas County, Texas",84.0,"21,928",32275,1392954,1194085,2037627,1676011,1326753,1291395,1.166545
2,"Denton County, Texas",94.5,"2,249",43250,579415,244764,662965,547898,424882,411328,2.367239
3,"Ellis County, Texas",93.1,623,37201,130247,41512,135260,111660,87970,85650,3.137575
4,"Hunt County, Texas",93.7,634,33664,64565,26438,75087,61557,48201,45671,2.442129
5,"Johnson County, Texas",94.7,925,37424,124558,39913,131321,109599,83170,84131,3.120738
6,"Kaufman County, Texas",95.5,682,36480,97537,23768,94327,78533,63092,59791,4.103711
7,"Parker County, Texas",96.3,520,35129,106949,24700,106767,90367,66140,67323,4.329919
8,"Rockwall County, Texas",97.0,382,50215,81035,15124,75858,63099,49144,47644,5.358040
9,"Tarrant County, Texas",90.5,"13,827",36216,1300652,728517,1600866,1317559,1047730,1006745,1.785342


In [88]:
# Final data frame for analysis and graphs
Final_df = Merged_3_df[['County','All Students','Family Violence Incidents','Median Earnings','Home Ownership Ratio']]

# Sets index to counties
Final_df = Final_df.set_index('County')
Final_df

,All Students,Family Violence Incidents,Median Earnings,Home Ownership Ratio
County,,,,
"Collin County, Texas",96.8,"3,227",50212,2.207404
"Dallas County, Texas",84.0,"21,928",32275,1.166545
"Denton County, Texas",94.5,"2,249",43250,2.367239
"Ellis County, Texas",93.1,623,37201,3.137575
"Hunt County, Texas",93.7,634,33664,2.442129
"Johnson County, Texas",94.7,925,37424,3.120738
"Kaufman County, Texas",95.5,682,36480,4.103711
"Parker County, Texas",96.3,520,35129,4.329919
"Rockwall County, Texas",97.0,382,50215,5.358040
